## Testa modellen på alla testbilder
**Mål:**
- Ladda den tränade modellen och köra den på testbilder
- Se felklassificerade bilder
- Analysera resultaten

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os
import shutil
import numpy as np
import psutil

# Kolla batteristatus
battery = psutil.sensors_battery()
if battery and battery.power_plugged is False:
    print("⚠️ Warning: Your laptop is running on battery. Performance may be reduced, "
          "and CUDA might fail due to power-saving features.")

# Check CUDA availability
if torch.cuda.is_available():
    print(f"✅ CUDA is available: Running on {torch.cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available: Running on CPU. Ensure your GPU drivers are installed correctly.")

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ Using device: {device}')

# Ladda modellen och sätt i eval-mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False)
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)
model.load_state_dict(torch.load('fire_classifier.pth', map_location=device))
model.to(device)
model.eval()

# Definiera testmapp
test_folders = {
    'fire': 'dataset/fire_images',
    'non_fire': 'dataset/non_fire_images'
}
# ✅ Definiera samma transform som vid träning
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Anpassa storleken till MobileNetV2s förväntade input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisering enligt MobileNetV2
])

## Gå igenom alla brandbilder och inte brandbilder

In [ ]:
true_labels = []
pred_labels = []
misclassified = []

# Kör inferens på alla testbilder
for label, folder in test_folders.items():
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)

        # Ladda & preprocessa bilden
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)

        # Gör en prediktion
        with torch.no_grad():
            output = model(image)
            probabilities = F.softmax(output, dim=1)
            confidence, predicted_class = torch.max(probabilities, 1)

        # Konvertera etiketter till numeriskt format för confusion matrix
        label_numeric = 1 if label == 'fire' else 0
        predicted_numeric = predicted_class.item()

        # Lagra resultat
        true_labels.append(label_numeric)
        pred_labels.append(predicted_numeric)

        # Om felklassificerad, spara i lista
        if predicted_numeric != label_numeric:
            predicted_label = ""
            if predicted_numeric == 1:
                predicted_label = "fire"
            else:
                predicted_label = "non_fire"
            misclassified.append((img_path, label, 'fire' if predicted_numeric == 1 else 'non_fire', confidence.item() * 100))
            print(f'❌ {img_path} | True: {label} | Pred: {predicted_label} ({confidence.item() * 10:.2f}%)')
print(f"✅ Testning klar! Totalt {len(true_labels)} bilder analyserade.")
print(f"❌ Felklassificerade bilder: {len(misclassified)}")


## 📊 Visa Confusion Matrix

# 📊 Förvirringsmatris (Confusion Matrix)

## 🔍 Vad är en Confusion Matrix?
En **Confusion Matrix** är ett verktyg för att **utvärdera** hur bra en AI-modell presterar på en **klassificeringsuppgift**.  
Den visar **hur många gånger** modellen hade **rätt** respektive **fel** när den försökte gissa rätt klass.

---

## 📝 Exempel på en Confusion Matrix

Låt oss säga att vi tränar en AI-modell för att **upptäcka brand** i bilder.

|   | **AI säger Brand 🔥** | **AI säger Ingen Brand ❌** |
|---|------------------|--------------------|
| **Verkligen Brand 🔥**  | **7** (Rätt) ✅ | **3** (Fel) ❌ |
| **Verkligen Ingen Brand ❌** | **2** (Fel) ❌ | **8** (Rätt) ✅ |

- ✅ **7 bilder med brand** klassificerades **korrekt**  
- ❌ **3 bilder med brand** klassificerades **felaktigt som ingen brand**  
- ❌ **2 bilder utan brand** klassificerades **felaktigt som brand**  
- ✅ **8 bilder utan brand** klassificerades **korrekt**  

---

## 📈 Hur tolkar vi en Confusion Matrix?
1. **Hög diagonal = Bra modell**  
   - Om AI:n klassificerar rätt ofta ser vi **höga värden på diagonalen** (övre vänstra till nedre högra hörnet).

2. **Höga siffror utanför diagonalen = Problem!**  
   - Om AI:n gör **många felklassificeringar**, har vi höga siffror **utanför diagonalen**.  
   - Exempel: Om **många bränder klassas som "ingen brand"**, kan AI:n vara **farligt opålitlig**.

---

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Skapa confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
class_names = ['non_fire', 'fire']

# Beräkna accuracy
accuracy = (cm[0, 0] + cm[1, 1]) / cm.sum() * 100

# Plotta confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title(f"Confusion Matrix: Test Set\nAccuracy: {accuracy:.2f}%")  # Lägg till accuracy i titeln
plt.show()

## ❌ Visa Felklassificerade Bilder
**Vad gick fel?**
- Visa några bilder som modellen klassificerade fel
- Jämför dem med en **lokal LLM (t.ex. Llama 3.2 Vision)**

In [ ]:
from IPython.display import display

# Visa de 20 första felklassificerade bilderna
for img_path, true_label, predicted_label, confidence in misclassified[:20]:
    print(f'❌ {img_path} | True: {true_label} | Pred: {predicted_label} ({confidence:.2f}%)')
    display(Image.open(img_path))

## 🤖 Låt Llama 3.2 Vision analysera samma bilder
Kan en **LLM förstå bilden bättre** än vår specialtränade AI?

In [ ]:
import ollama
import textwrap

# Testa en bild med Llama 3.2 Vision
llama_image = misclassified[0][0]  # Första felklassificerade bilden

response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image? Please provide reasoning step by step.',
        'images': [llama_image]
    }]
)

# Formatera outputen
wrapped_response = textwrap.fill(response['message']['content'], width=80)

print(f'📸 Bild: {llama_image}')
display(Image.open(llama_image))
print('\n🧠 Llama 3.2 Vision Response:\n')
print(wrapped_response)